# Elasticstack
Úvodem bych rád poznamenal, že tento notebook slouží primárně jako soukromá sbírka poznámek. Není to podklad pro nějaký workshop, neboť na něco takového mi v době psání těchto řádků zkrátka chybí zkušenosti.

## Obsah
[Terminologie](#Terminologie)  
[Elasticsearch](#Elasticsearch)  
[\_\_Indexy](#Indexy)  
[\_\_Vkládání dokumentů do indexu](#Vkládání-dokumentů-do-indexu)  
[\_\_Vyhledávání v dokumentech](#Vyhledávání-v-dokumentech)  
[\_\_\_\_Query](#Query)  
[\_\_\_\_\_\_match a match_phrase](#match-a-match_phrase)  
[\_\_\_\_\_\_multi_match](#multi_match)  
[\_\_\_\_\_\_bool](#bool)  
[\_\_\_\_Agregace](#Agregace)  
[\_\_Updaty a mazání záznamů](#Updaty-a-mazání-záznamů)  
\_\_[Elastic a curl](#Elastic-a-curl)  
[\_\_Elastic a Python](#Elastic-a-Python)  


## Terminologie
Elasticsearch sice umožňuje ukládat strukturovaná data, spíše než o databázi se ale jedná o search engine. To na jedné straně znamená, že je Elastic ideální pro full textová prohledávání, na druhé straně ale například nepodporuje joinování či transakce. Nicméně určité paralely mezi ním a klasickou databází se najít dají.  
Základním stavebním kamenem, ekvivalentním řádku v tabulce, je dokument. Tento objekt je reprezentován jsonem. Na základě logiky dat se pak dokumenty sdružují do indexů. U těchto indexů ale narozdíl od databázových tabulkek nic nevynucuje schéma. Tj. různé dokumenty-jsony umístěné v témže indexu mohou mít různé klíče a mohou mít i různý počet klíčů.  
Měli bychom poznamenat, že indexy v sobě reálně jednotlivé dokumenty neobsahují. To, co se v indexech nachází, jsou *reference* na dokumenty. Fakticky jsou data dokumentů uloženy v tzv. shardech. O co se jedná a jak vůbec vypadá HW hierarchie Elasticu? Nejvýše stojí cluster. Pod ním se nacházejí ve vztahu one-to-many jednotlivé nody, bydlící obvykle na různých serverech. Na jednotlivých nodech jsou pak umístěny shardy. Aby se při pádu serveru neztratila data, má každý primární shard několik replik bydlících na odlišných nodech, které primární shard v případě nutnosti zastoupí.  
Když je vytvořen nový index, defaultně se vytvoří nový shard. Ten postupně roste a tak by mohla nastat situace, kdy by byl onen shard větší než prostor na nodu. Proto se v takovém případě na jiném nodu vytvoří nový shard, do něhož se další dokumenty spadající pod stejný index budou ukládat. Jednou z výhod takového řešení je i skutečnost, že se takto zmenší doba zpracování dotazů. Pokud by totiž data byla rozdělena ve dvou shardech bydlících na dvou různých nodech, ony nody data prohledají rychleji, než by to zvládl node jeden zkoumající data všechna.

Elasticstack se skládá z následujících částí:
- Elasticsearch - search engine alias jádro Elasticu
- Kibana - webové rozhraní umožňující vizualizovat data
- Logstash - data procesující pipelina

## Elasticsearch

Nejprve musíme spustit Elasticsearch (elasticsearch-7.13.3/bin/elasticsearch.bat) coby jádro vyhledávacího enginu a posléze Kibanu (kibana-7.13.3-windows-x86_64/bin/kibana.bat), jejímž prostřednictvím budeme s elasticem primárně pracovat. Defaultně se Elasticsearch nalézá na localhost:9200 a Kibana na localhost:5601. Vlezeme právě na kibaní URL a jakmile se v prohlížeči Kibana načte, klikneme na tři vodorovné čáry vlevo nahoře a v menu vybereve "Dev Tools (je až úplně dole v sekci "Management"). Otevře se konzole, kde v levé čáasti zadáváme příkazy (spouští se trojúhelníkem na pravém okraji levé sekce) a v pravé sekci se pak díváme na výstupy.  
### Indexy
Když chceme vytvořit nový index, máme na výběr několik možných přístupů. Nejkratší z nich je asi
```
PUT jmeno-indexu
```
tedy např.
```
PUT pokusny_index_1
```
V takovémto případě se vrátí zpráva
```
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "pokusny_index_1"
}
```
V rámci PUT requestu máme možnost specifikovat pole i jejich typy. 
```
PUT pokusny_index_3
{
  "mappings":{
    "properties": {
      "jmeno_produktu":{"type":"text"},
      "pocet":{"type":"long"},
      "cena":{"type":"float"}
    }
  }
}
```
Nicméně pozor - pokud do takovéhoto indexu posléze zkusíme vložit záznam s jinými poli, projde to - schéma se o nová pole zkrátka rozšíří. Pouze v případě, kdy bychom do již specifikovaného pole zkoušeli vložit nekompatibilní obsah (např. do čísleného pole "cena" text "nazdar"), vrátila by se nám chyba 
```
{
  "error" : {
    "root_cause" : [
      {
        "type" : "mapper_parsing_exception",
        "reason" : "failed to parse field [cena] of type [float] in document with id 'O9QDSnsBNZx_bel0v5mI'. Preview of field's value: 'nazdar'"
      }
    ],
    "type" : "mapper_parsing_exception",
    "reason" : "failed to parse field [cena] of type [float] in document with id 'O9QDSnsBNZx_bel0v5mI'. Preview of field's value: 'nazdar'",
    "caused_by" : {
      "type" : "number_format_exception",
      "reason" : "For input string: \"nazdar\""
    }
  },
  "status" : 400
}

```
Datových typů existuje celá řada. Nicméně pro začátek je asi nejdůležitější naučit se rozlišovat mezi dvěma určenými pro textové řetězce - "text" a "keyword". Zdůrazněme, že na jedno pole mohou být napojené oba typy, nicméně počítejme s tím, že pro importovaný dataset (viz dále) si elastic obvykle vybere buďto jeden, nebo druhý typ. Text se používá na full-text prohledávání, zatímco keyword se hodí pro přesné vyhledávání termínů, sortování a agregace. Pakliže je pole označeno jako text, jeho obsah projde analyzátorem, který zajistí tokenizaci, lowercasování a odstranění interpunkce. Navíc se vytvoří tzv. "inverted index", který mapuje slova na id dokumetu. To má poté využití v dotazech typu "Ukaž všechny dokumenty obsahující slova X, Y a Z". I keyword search má svou vlastní meta-tabulku - tzv. "doc values". Zde je na každém řádku id dokumentu a termín. Pokud se termín objeví v jiném dokumentu, je z toho nový řádek, nikoli další položka v buňce id dokumentu, jako je tomu u inverted indexu.  

Další možností pro vytvoření indexu je využít POST request, kdy v rámci jednoho příkazu index vytvoříme i naplníme prvním záznamem. Obecný předpis je
```
POST jmeno-indexu/_doc
{
  "jmeno_pole": hodnota_pole
}
```
Konkrétní příklad:
```
POST pokusny_index_2/_doc
{
  "prvni_pole": 1,
  "druhe_pole": 100
}
```
Vrátí se zpráva o následujícím obsahu:
```
{
  "_index" : "pokusny_index_2",
  "_type" : "_doc",
  "_id" : "N9TsSXsBNZx_bel0upmB",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 0,
  "_primary_term" : 1
}
```

Pokud chceme vidět "schéma" indexu, použijeme 
```
GET jmeno_indexu
```
Resp. pokud bychom chtěli opravdu jen mapping bez dodatečných informací, použili bychom 
```
GET historic_events/_mapping
```
Pro index, který zatím schéma nikterak definované nemá, vrátí "krátký" GET následující
```
{
  "pokusny_index_1" : {
    "aliases" : { },
    "mappings" : { },
    "settings" : {
      "index" : {
        "routing" : {
          "allocation" : {
            "include" : {
              "_tier_preference" : "data_content"
            }
          }
        },
        "number_of_shards" : "1",
        "provided_name" : "pokusny_index_1",
        "creation_date" : "1629032521954",
        "number_of_replicas" : "1",
        "uuid" : "et9_E9CXQJuIyV4OtEEBlA",
        "version" : {
          "created" : "7130399"
        }
      }
    }
  }
}
```
Pro index s již vytvořeným schématem dostaneme toto
```
{
  "pokusny_index_2" : {
    "aliases" : { },
    "mappings" : {
      "properties" : {
        "druhe_pole" : {
          "type" : "long"
        },
        "prvni_pole" : {
          "type" : "long"
        }
      }
    },
    "settings" : {
      "index" : {
        "routing" : {
          "allocation" : {
            "include" : {
              "_tier_preference" : "data_content"
            }
          }
        },
        "number_of_shards" : "1",
        "provided_name" : "pokusny_index_2",
        "creation_date" : "1629032852067",
        "number_of_replicas" : "1",
        "uuid" : "d5UL-uBJT3qjaTz_sKQ4IA",
        "version" : {
          "created" : "7130399"
        }
      }
    }
  }
}
```

Zmiňme ještě, že jednou vytvořené indexy se nedají měnit. Tj. pokud bychom u nějak specifikovaného pole chtěli změnit typ hodnoty, která je v něm uložena, musíme vytvořit index nový a data do něj přelít.

Co se týče mazání indexu, syntax je následující:
```
DELETE jmeno-indexu
```
Návratová hodnota má podobu
```
{
  "acknowledged" : true
}
```

Nakonec ještě uveďmě dotaz zobrazující přehled již vytvořených indexů:
```
GET _cat/indices
```

### Vkládání dokumentů do indexu
Dokument můžeme do indexu vložit buďto pomocí POSTu, jako jsme to už udělali výše, anebo pomocí PUTu.
Výše uvedenou POSTovskou syntax můžeme použít i když chceme vložit dokument do již existujícího indexu.
```
POST jmeno-indexu/_doc
{
  "jmeno_pole": hodnota_pole
}
```
Když se podíváme na návratovou zprávu, vidíme, že se pro dokument vytvořilo automaticky IDčko.
```
{
  "_index" : "pokusny_index_2",
  "_type" : "_doc",
  "_id" : "VBJhS3sBihumCpkYEcp1",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 2,
  "_primary_term" : 2
}
```
Nicméně IDčko můžeme specifikovat i explicitně a to tak, že za \_doc přidáme dopředné lomítko a ID, tj.
```
POST jmeno-indexu/_doc/zvolene_idcko
{
  "jmeno_pole": hodnota_pole
}
```
Pakliže ale na to samé IDčko pošleme jiný záznam, provede se update - nahrazení starého záznamu novým. V návratové zprávě se pole "result" z "created" změní na "updated" a číslo u položky "\_version" se zvýší o jedna.
```
{
  "_index" : "pokusny_index_2",
  "_type" : "_doc",
  "_id" : "5",
  "_version" : 3,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 7,
  "_primary_term" : 2
}
```

Pakliže si chceme být jisti, že k přepisu stávajícího dokumentu nedopatřením nedojde, musíme použít lehce odlišný endpoint - namísto "\_doc" aplikujeme "\_create", například takto:
```
POST pokusny_index_2/_create/6
{
  "prvni_pole": 6,
  "druhe_pole": 600
}
```
Pokud není příslušné IDčko zabrané, dokument se uloží. Pokud už ale dané IDčko používáno je, vrátí se návratový kód 409 a chybová zpráva
```
{
  "error" : {
    "root_cause" : [
      {
        "type" : "version_conflict_engine_exception",
        "reason" : "[6]: version conflict, document already exists (current version [1])",
        "index_uuid" : "d5UL-uBJT3qjaTz_sKQ4IA",
        "shard" : "0",
        "index" : "pokusny_index_2"
      }
    ],
    "type" : "version_conflict_engine_exception",
    "reason" : "[6]: version conflict, document already exists (current version [1])",
    "index_uuid" : "d5UL-uBJT3qjaTz_sKQ4IA",
    "shard" : "0",
    "index" : "pokusny_index_2"
  },
  "status" : 409
}
```

Do jisté míry se POSTu podobá PUT. Jeho základní syntaxí je 
```
PUT jmeno-indexu/_doc/zvolene_idcko
{
  "jmeno_pole": hodnota_pole
}
```
Stejně jako v POSTu funguje výměna \_doc za \_create. Co ale nefunguje je použití PUTu bez specifikování IDčka - tehdy obdržíme chybu 
```
{
  "error" : "Incorrect HTTP method for uri [/pokusny_index_2/_doc?pretty=true] and method [PUT], allowed: [POST]",
  "status" : 405
}
```
Obecně by asi bylo nejčistější použít POST na vytvoření nového dokumentu a PUT na jeho případný update.

Zatím jsme řešili manuální vkládání dokumentů do elasticu jeden po druhém. Kibana ale dovoluje vložit najednou celý soubor. To se provádí tlačítkem "Upload a file" na úvodní stránce Kibany (tlačítko je pod velkou modrou Kibana dlaždicí). Sluší se podotknout, že i když elastic dokumenty ukládá jako jsony, srovná se i s tím, když mu člověk podhodí csvčko (data si na json sám zkonvertuje). Nicméně je zde jedna zrada - jsony musí být ve formátu ndjson. To znamená, že na každém řádku je validní json a tyto jsony jsou odděleny pouze znakem nového řádku (tj. žádná čárka). Příkladem budiž toto:
```
{"date": "-300", "description": "Pilgrims travel to the healing temples of Asclepieion to be cured of their ills. After a ritual purification the followers bring offerings or sacrifices.", "lang": "en", "category1": "By place", "category2": "Greece", "granularity": "year"}
{"date": "-300", "description": "Pyrrhus, the King of Epirus, is taken as a hostage to Egypt after the Battle of Ipsus and makes a diplomatic marriage with the princess Antigone, daughter of Ptolemy and Berenice.", "lang": "en", "category1": "By place", "category2": "Egypt", "granularity": "year"}
{"date": "-300", "description": "Ptolemy concludes an alliance with King Lysimachus of Thrace and gives him his daughter Arsinoe II in marriage.", "lang": "en", "category1": "By place", "category2": "Egypt", "granularity": "year"}
```

### Vyhledávání v dokumentech 

Pro příklady vyhledávání v dokumentech použijeme dataset "Historical events" [https://github.com/jdorfman/awesome-json-datasets#historical-events](odtud). Je zde ale drobná komplikace - tento json není ve formátu ndjson. Aby ho elastic byl schopen zpracovat, musí se jednak ze začátku souboru odstranit
```
{"result": {"count": "37859" 
```
a dále musí zmizet dvě uzavírající složené závorky na konci souboru. Hlavně ale musí být každý výskyt
```
, "event": 
```
nahrazen znakem nového řádku.  
Pro čtení záznamů existují dva endpointy - \_doc a \_search. Význam \_doc spočívá v tom, že nám umožňuje najít záznam na základě jeho id - připomeňme, že pokud id nezadáme explicitně při vzniku záznamu, vytvoří si ho Elastic sám. A to i při nahrávání souboru s hromadou záznamů. Podoba requestu je následující
```
GET jmeno-indexu/_doc/zvolene-idcko
```
v praxi tedy například
```
GET historic_events/_doc/gNEWansB2kiDSx5Po1A5
```
přičemž výsledkem je
```
{
  "_index" : "historic_events",
  "_type" : "_doc",
  "_id" : "gNEWansB2kiDSx5Po1A5",
  "_version" : 1,
  "_seq_no" : 0,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "date" : "-300",
    "description" : "Pilgrims travel to the healing temples of Asclepieion to be cured of their ills. After a ritual purification the followers bring offerings or sacrifices.",
    "lang" : "en",
    "category1" : "By place",
    "category2" : "Greece",
    "granularity" : "year"
  }
}
```
Search endpoint má využití širší. Jeho použití bez čehokoli dalšího vede k ukázání základních statistických informací a (defaultně) deseti prvních záznamů z indexu. Tj. člověk zadá např.
```
GET historic_events/_search
```
a vrátí se mu následující (pro přehlednost jsou druhý až desátý záznam odstraněny)
```
{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 10000,
      "relation" : "gte"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "historic_events",
        "_type" : "_doc",
        "_id" : "gNEWansB2kiDSx5Po1A5",
        "_score" : 1.0,
        "_source" : {
          "date" : "-300",
          "description" : "Pilgrims travel to the healing temples of Asclepieion to be cured of their ills. After a ritual purification the followers bring offerings or sacrifices.",
          "lang" : "en",
          "category1" : "By place",
          "category2" : "Greece",
          "granularity" : "year"
        }
      },
...
    ]
  }
}
```
Co se to vlastně v úvodu odpovědi objevuje? Took říká, kolik milisekund zpracování dotazu na straně Elasticu zabralo. Time_out odpoví na dotaz, zda došlo k time outu. Sekce \_shards se týká toho, s kolika shardy (a s jakým úspěchem) se při práci na dotazu muselo pracovat. V sekci "hits" máme položku value. Ta by měla říkat, kolik záznamu odpovídá požadavkům v search requestu. Nicméně 10000 je až podezřele kulaté číslo. Jak moc mu můžeme věřit říká položka "relations". Pokud je rovna "eq", je hodnota "value" přesná, pokud má ale hodnotu "gte", je faktický počet záznamu větší než (greather than) číslo ve "value". Pokud bychom chtěli znát přesný počet záznamů u větších datasetů, museli bychom použít flag "track_total_hits":
```
GET historic_events/_search
{
  "track_total_hits": true
}
```
Poté by v sekci "hits" bylo něco ve stylu
```
    "total" : {
      "value" : 37858,
      "relation" : "eq"
    }
```

#### Query
##### match a match_phrase
Pro vyhledávání slov a frází se dá použít kombinace klíčových slov query a match resp. match_phrase. Template pro syntax je následující 
```
GET jmeno-indexu/_search
{
  "query": {
    "match":{
      "jmeno-prohledavaneho-pole":{
        "query": "hledana slova"
      }
    }
  }
}
```
V praxi to vypadá takto:
```
GET historic_events/_search
{
  "query": {
    "match":{
      "description":{
        "query": "Roman empire"
      }
    }
  }
}
```
Poznamenejme, že matchnuty jsou záznamy, které obsahují aspoň jedno slovo z druhé "query", a že velikost písmen nehraje roli.  
Všimněme si, že výstup - prvních X záznamů - je seřazen podle \_score. Veličina \_score zde představuje více méně TF-IDF metriku (viz notebook o zpracování textu v tomto repozitáři). Pakliže bychom hledali namísto "Roman empire" řetězec "empire Roman", výsledek by byl totožný. To je umožněno tím, že pole "description" má datový typ "text". Pokud bychom stejnou věc chtěli vyzkoušet u pole "category2" mající typ keyword, tak by byl v druhém případě počet odpovídajících dokumentů 0.  
Pokud chceme vidět problémy této query, nesmíme koukat na záznamy s nějvětším skore, ale na ty se skorem nejnižším. Tj. musíme podle \_score provést ascending sortění. To uskutečníme tak, že na úroveň query dáme další klíčové slovo a to sice "sort":
```
GET historic_events/_search
{
  "query": {
    "match":{
      "description":{
        "query": "Roman Empire"
      }
    }
  },
  "sort": { "_score": { "order": "asc" }}
}
```
Vidíme, že záznamy s nízkým skore se Římského impéria netýkají. Nicméně jelikož se v nich vyskytují slova "Roman" a "Empire" (často v jiných větách, někdy dokonce jedno z nich absentuje), jsou tyto dokumenty stále matchnuty. Pokud ale chceme nalézt jen dokumenty určitou frázi (např. "Roman empire") obsahující, musíme "match" zaměnit za "match_phrase".
```
GET historic_events/_search
{
  "query": {
    "match_phrase":{
      "description":{
        "query": "Roman Empire"
      }
    }
  },
  "sort": { "_score": { "order": "asc" }}
}
```
Vidíme, že počet záznamů klesnul na cca desetinu, ale i ty záznamy s nejnižším skorem sousloví "Roman empire" obsahují (i když se jedná o Svatou říši římskou, což je přeci jen jiný státní útvar). Samozřejmě u match_phrase prohození "Roman" a "Empire" povede k nulovému počtu nalezených dokumentů.  
Vraťme se ale od "match_phrase" zpět k "match". Viděli jsme, že při zpracování slov v poli "query" jsou tato slova oddělena podle mezer a následně se matchnou dokumenty obsahující aspoň jedno z těchto slov. Co ale máme dělat v případě, kdy chceme matchnut pouze dokumenty obsahující *všechna* slova z query, avšak netrváme na tom, aby se tato slvoa nacházela těsně vedle sebe? V takovém případě obohatíme dotaz o pole "operator" s hodnotou "and", které bude na stejné úrovni jako druhé "query":
```
GET historic_events/_search
{
  "query": {
    "match":{
      "description":{
        "query": "Roman Empire",
        "operator": "and"
      }
    }
  },
  "sort": { "_score": { "order": "asc" }}
}
```
Existuje ještě třetí cesta, kdy specifikujeme, že chceme dokumenty obsahující aspoň X slov z query. Pro to použijeme pole "minimum_should_match" mající za hodnotu právě ono číslo X.
```
GET historic_events/_search
{
  "query": {
    "match":{
      "description":{
        "query": "Roman Empire republic",
        "minimum_should_match": "2"
        
      }
    }
  },
  "sort": { "_score": { "order": "asc" }}
}
```

##### multi_match
Může natat situace, kdy budeme slova hledat ve více polích. Tehdy namísto matche použijeme multi_match. Základní struktura takového dotazu vypadá takto:
```
GET jmeno-indexu/_search
{
  "query": {
    "multi_match": {
        "query": "chtena slova",
        "fields": ["prvni_pole", "druhe_pole"]
    }
  }
}
```
Praktický příklad:
```
GET historic_events/_search
{
  "query": {
    "multi_match": {
        "query": "Hamburg Prussia",
        "fields": ["category2", "description"]
    }
  }
}
```
Může se stát, že budeme chtít dát jednomu poli větší váhu než polím jiným. To zrealizujeme tak, že v sekci "fields" vložíme za jméno pole "umocňovátko" (stříšku) a za to stupeň "umocnění" (stříška i číslo bude pořád v dvojitých závorkách příslušejících jménu pole).
```
GET historic_events/_search
{
  "query": {
    "multi_match": {
        "query": "Hamburg Prussia",
        "fields": ["category2^2", "description"]
    }
  }
}
```
Co ale v případě, kdy potřebujeme ve více polích hledát fráze? Nemusíme vymýšlet žádné multi_match_phrase, pořád nám vystačí multi_match, pouze do něj přidáme nové pole "type" s hodnotou "phrase".
```
GET historic_events/_search
{
  "query": {
    "multi_match": {
        "query": "Roman Republic",
        "fields": ["category2^2", "description"],
        "type": "phrase"
    }
  }
}
```

##### bool
Co když ale chceme ještě složitější dotazy - například co když požadujeme přítomnost uručtého slova v jednom poli a přítomnost slova jiného v jiném poli? Tehdy musíme použít bool query. Podsekcí boolu bude "must", které v sobě může obsahovat jak "match", tak "match_phrase". Platí přitom, že vnitřnosti "must" jsou v hranatých závorkách.
```
GET jmeno-indexu/_search
{
  "query": {
    "bool": {
      "must": [
        {"match_phrase": {
          "jmeno_pole": "fraze"
        }},
        {"match": {
          "jmeno_jineho_pole": "hledana slova"
        }}
      ]
    }
  }
}
```
Praktický příklad:
```
GET historic_events/_search
{
  "query": {
    "bool": {
      "must": [
        {"match_phrase": {
          "description": "civil war"
        }},
        {"match": {
          "category2": "Roman Empire"
        }}
      ]
    }
  }
}
```
Současně lze i specifikovat slova/fráze, která ve vrácených dokumentech být nemají:
```
GET historic_events/_search
{
  "query": {
    "bool": {
      "must": [
        {"match_phrase": {
          "description": "Roman Empire"
        }}
      ],
      "must_not": [
         {"match_phrase": {
          "description": "Roman Republic"
        }}
      ]
    }
  }
}
```
K "must" a "must_not" je příbuzné klíčové slovo "should". Slova a fráze v "should" nejsou povinná, nicméně dokuemtny je obsahující dostanou vyšší skore.
```
GET historic_events/_search
{
  "query": {
    "bool": {
      "must": [
        {"match_phrase": {
          "description": "Roman Republic"
        }}
      ],
      "should": [
         {"match": {
          "description": "war"
        }}
      ]
    }
  }
}
```

#### Agregace
Někdy nepotřebujeme znát obsah záznamů vyhovujících nějaké podmínce, ale musíme vědět, kolik takových záznamů v daném indexu vlastně existuje. Případně chceme spočítat průměr, sumu či nějakou další metriku odvoditelnou z hodnot nějakého sloupce. Tehdy mluvíme o agregacích.  
Pro spočítání záznamů s určitou hodnotou sloupce se použije předpis
```
GET jmeno-indexu/_search
{
  "aggs": {
    "nazev_sekce_odpovedi": {
      "terms": {
        "field": "jmeno-sloupce-podle-ktereho-se-agreguje"
      }
    }
  }
}
```
Praktický příklad:
```
GET historic_events/_search
{
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "category2"
      }
    }
  }
}
```
Odpověď Elasticu se skládá ze tří částí. Nejprve jsou zobrazena metadata, potom je v sekci "hits" defaultně ukázáno 10 záznamů z indexu až poté se k sekci "aggregations" objeví deset nejzastoupenějších záznamů. No, jo, ale co dělat, když "hits" zobrazovat nechceme a počet ukázaných zagregovaných tříd si chceme nastavit sami?  
Abychom se zbavili ukázek záznamů, musíme před "aggs" na stejnou úroveň přidat parametr "size" s hodnotou 0:
```
GET historic_events/_search
{
  "size": 0,
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "category2"
      }
    }
  }
}
```
Pokud chceme specifikovat maximální počet ukázaných tříd, opět použijeme parametr "size", který ale tentokrát umístíme na úroveň pole "field":
```
GET historic_events/_search
{
  "size": 0,
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "category2",
        "size": 3
      }
    }
  }
}
```
Výsledkem je pak následující odpověď Elasticu:
```
{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 10000,
      "relation" : "gte"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "pocty_zaznamu" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 4078,
      "buckets" : [
        {
          "key" : "Europe",
          "doc_count" : 2182
        },
        {
          "key" : "Roman Empire",
          "doc_count" : 1820
        },
        {
          "key" : "Asia",
          "doc_count" : 1506
        }
      ]
    }
  }
}
```
Dosti rozumný je požadavek, aby agregace proběhly na už předfiltrované množině dat. Jazykem SQLka to znamená použít GROUP BY současně s WHERE podmínkou. Například pokud bychom z nějakého záhadného důvodu chtěli odfiltrovat všechny třídy až na jednu a nad výsledkem této selekce udělat agregaci, provedli bychom to takto:
```
GET historic_events/_search
{
  "query": {
    "match": {
      "category2": "Roman Republic"
    }
  },
  "size": 0,
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "category2",
        "size": 30
      }
    }
  }
}
```
Jelikož index s historickými událostmi neobsahuje žádná čísla, u kterých by artimetické operace dávaly smysl, budeme chvíli pracovat s indexem iris_from_csv. Pokud chceme spočítat průměrnou hodnotou nějakého sloupce, použijeme následující šablonu:
```
GET jmeno-indexu/_search
{
  "size": 0,
  "aggs": {
    "nazev_sekce_odpovedi": {
      "avg": {
        "field": "jmeno-sloupce-podle-ktereho-se-agreguje"
      }
    }
  }
}
```
Praktická ukázka:
```
GET iris_from_csv/_search
{
  "size": 0,
  "aggs": {
    "prumer_sepa_width": {
      "avg": {
        "field": "sepal_width"
      }
    }
  }
}
```
Na místě "avg" by mohly být i "sum", "min", "max" či třeba "cardinality" (ukáže počet unikátních hodnot). Pokud bychom chtěli najednou ukázat počet záznamů, jejich sumu a nejmenší, největší a  průměrný záznam, použili bychom "stats". Pakliže ale chceme mít vedle sebe jen některé agregace (které ani nemusí být součástí "stats"), umístíme bloky dvou pojmenovaných agregací na stejnou úroveň. Na příkladu níže se jednak počítá výskyt hodnot ve sloupečku "species", jednak se (přes všechny "species") spočítá průměrný "sepal_length":
```
GET iris_from_csv/_search
{
  "size": 0,
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "species"
      }
    },
    "prumer_sepal_length": {
      "avg": {
        "field": "sepal_length"
      }
    }
  }
}
```
Takto vypadá výsledek:
```
{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 149,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "pocty_zaznamu" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : "setosa",
          "doc_count" : 50
        },
        {
          "key" : "versicolor",
          "doc_count" : 50
        },
        {
          "key" : "virginica",
          "doc_count" : 49
        }
      ]
    },
    "prumer_sepal_length" : {
      "value" : 5.842953020134228
    }
  }
}
```
Pokud bychom ale chtěli spočítat průměrnou "sepal_length" pro každý typ kosatce, musíme do prvního "aggs" vložit druhé "aggs" na úroveň prvního uživatelem pojmenovaného pole:
```
GET iris_from_csv/_search
{
  "size": 0,
  "aggs": {
    "pocty_zaznamu": {
      "terms": {
        "field": "species"
      },
      "aggs": {
        "prumer_sepal_length": {
          "avg": {
            "field": "sepal_length"
          }
        }
      }
    }
  }
}
```
Výsledná odpověď:
```
{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 149,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "pocty_zaznamu" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : "setosa",
          "doc_count" : 50,
          "prumer_sepal_length" : {
            "value" : 5.006
          }
        },
        {
          "key" : "versicolor",
          "doc_count" : 50,
          "prumer_sepal_length" : {
            "value" : 5.936
          }
        },
        {
          "key" : "virginica",
          "doc_count" : 49,
          "prumer_sepal_length" : {
            "value" : 6.6020408163265305
          }
        }
      ]
    }
  }
}
```
Proberme ještě range agregaci. Ta funguje tím způsobem, že uživatel specifikuje sloupec a intervaly hodnot v něm a Elastic spočítá, kolik záznamů patří do jakého intervalu:
```
GET iris_from_csv/_search
{
  "size": 0,
  "aggs": {
    "range_sepal_length": {
      "range": {
        "field": "sepal_length",
        "ranges": [
          {
            "to": 2
          },
          {
            "from": 2.01,
            "to": 5
          },
          {
            "from": 5.01
          }
        ]
      }
    }
  }
}
```
Odpověď Elasticu:
```
{
  "took" : 102,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 149,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "range_sepal_length" : {
      "buckets" : [
        {
          "key" : "*-2.0",
          "to" : 2.0,
          "doc_count" : 0
        },
        {
          "key" : "2.01-5.0",
          "from" : 2.01,
          "to" : 5.0,
          "doc_count" : 22
        },
        {
          "key" : "5.01-*",
          "from" : 5.01,
          "doc_count" : 117
        }
      ]
    }
  }
}
```
Pokud nechceme specifikovat kraje intervalů, ale pouze jejich velikost, můžeme použít typ agregace zvaný histogram:
```
GET iris_from_csv/_search
{
  "size": 0,
  "aggs": {
    "histogram_sepal_length": {
      "histogram": {
        "field": "sepal_length",
        "interval": 1.5
      }
    }
  }
}
```
Odezva Elasticu:
```
{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 149,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "histogram_sepal_length" : {
      "buckets" : [
        {
          "key" : 3.0,
          "doc_count" : 4
        },
        {
          "key" : 4.5,
          "doc_count" : 78
        },
        {
          "key" : 6.0,
          "doc_count" : 61
        },
        {
          "key" : 7.5,
          "doc_count" : 6
        }
      ]
    }
  }
}
```

### Updaty a mazání záznamů
Pro updatování záznamů se použije předpis
```
POST jmeno-indexu/_update/id-dokumentu
{
  "doc":{
     "prvni_updatovane_pole":"prvni_updatovana_hodnota",
     "druhe_updatovane_pole":"druha_updatovana_hodnota"
  }
}
```
Například tedy:
```
POST pokusny_index_2/_update/5
{
  "doc": {
    "druhe_pole": 412
  }
}
```
Zdůrazněme, že pro PUT výše uvedený postup nefunguje a že pokud pole upravit nechceme, prostě ho v příkazu nezmíníme. Rozdíl \_update a \_doc spočívá v tom, že s pomocím druhého zmíněného endpointu se upraví celý záznam, zatímco \_update mění jen explicitně zmíněná pole.   
Mazaní se provede prostřednictvím
```
DELETE jmeno-indexu/_doc/id-dokumentu
```
Tj. třeba:
```
DELETE pokusny_index_2/_doc/5
```

### Elastic a curl
Curl je program spouštěný z příkazové řádky, který dokáže provolávat endpointy. To znamená, že jeho prostřednictvím můžeme pracovat s Elasticem i bez konzole v Kibaně.  
Asi nejjednodušší je dotázat se s jeho pomocí na obsah dokumentu, známe-li jeho IDčko. Tehdy stačí napsat
```
curl http://localhost:9200/historic_events/_doc/gNEWansB2kiDSx5Po1A5
```
a jako odpověď obdržíme
```
{"_index":"historic_events","_type":"_doc","_id":"gNEWansB2kiDSx5Po1A5","_version":1,"_seq_no":0,"_primary_term":1,"found":true,"_source": {"date": "-300", "description": "Pilgrims travel to the healing temples of Asclepieion to be cured of their ills. After a ritual purification the followers bring offerings or sacrifices.", "lang": "en", "category1": "By place", "category2": "Greece", "granularity": "year"}}
```
Co když ale chceme provést nějaké vyhledávání? Tehdy je řešením bohužel trochu nepřehledná variace na 
```
curl -XGET --header "Content-Type: application/json" http://localhost:9200/historic_events/_search -d "{\"query\" : {\"match\" : { \"description\": \"Roman empire\" }}}"
```
Zdůrazněme, že výše uvedený kód by měl být jednořádkový (ve smyslu neměl by obsahovat znak nového řádku). Všiměte si, že je většina dvojitých uvozovek escapovaná zpětným lomítkem - neescapované uvozovky totiž specifikují, kde začíná a končí jsony, a bohužel kombinování jednoduchých a dvojitých uvozovek minimálně na Windowsech k cíly nevede. Stejně tak nepomohla snaha použít k nalepení jednotlivých řádků na sebe prostřednictvím znaků stříšky.     
Co vlastně znamenají jednotlivé parametry curlu? Parametr -d prozrazuje, že za ním nacházející se věc jsou data. Header pak obsahuje informaci o tom, v jakém formátu ona data vlastně jsou. Jelikož posíláme json, musíme to zde i napsat, jinak bychom obdrželi chybu *{"error":"Content-Type header [application/x-www-form-urlencoded] is not supported","status":406}*.  

S parametrem -XGET je to trochu složitější. Provolání elasticu by totiž fungovalo i bez něho. Curl se totiž snaží na základě parametrů rozpoznat, o jaký typ requestu vlastně jde. Například první aplikace curlu na začátku této kapitoly žádný parametr neměla a tak curl usoudil, že se bude jednat o GET (můžeme ověřit přidáním nepovinného parametru --verbose). Nicméně u druhého příkladu si curl všiml, že používáme parametr -d, a tak usoudil, že se jedná o POST. Při aplikaci \_search endpointu naštěstí GET a POST dávají stejné výsledky (viz [dokumentace](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html)), ale už z hlediska konzistence s předcházející části bychom přeci jen raději použili GET. No a to obstarává parametr -X s hodnotou GET (alias -XGET).  

Pro přidání záznamu do indexu použijeme
```
curl --header "Content-Type: application/json" http://localhost:9200/pokusny_index_2/_doc -d "{\"prvni_pole\": 7, \"druhe_pole\": 700}"
```
Naopak pro smazání musíme použít volání
```
curl -XDELETE http://localhost:9200/pokusny_index_2/_doc/YSTVoXsB-GS4cL4lD-pI 
```

### Elastic a Python

Ačkoli pro Elastic existují v Pythonu dedikované knihovny, podíváme se zde na způsob, jak s tímto vyhledávacím enginem komunikovat standardním způsobem - prostřednictvím balíčku requests. 

In [1]:
import requests

Pro "bezparametrové" dotazy stačí zkonstruovat url, která bude předána jako parametr funkci get. Takto získaný výstup bude nejčitelnější při použití metody json. 

In [2]:
elasticsearch_url = "http://localhost:9200"
index_name = "historic_events"
document_id = "gNEWansB2kiDSx5Po1A5"
whole_url = elasticsearch_url + "/" + index_name + "/_doc/" + document_id
response = requests.get(whole_url)
response.json()

{'_index': 'historic_events',
 '_type': '_doc',
 '_id': 'gNEWansB2kiDSx5Po1A5',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'date': '-300',
  'description': 'Pilgrims travel to the healing temples of Asclepieion to be cured of their ills. After a ritual purification the followers bring offerings or sacrifices.',
  'lang': 'en',
  'category1': 'By place',
  'category2': 'Greece',
  'granularity': 'year'}}

Pro složitější dotazy bude potřeba jednak zkonstruovat query, která bude funkci *get* předána v parametru data, a jednak informaci o formátu dat, která se předá v parametru header. Zdůrazněme, že zatímco data pro hlavičku requestu mají podobu slovníku, query je reprezentována stringem, jehož vnitřek json pravda připomíná.

In [9]:
elasticsearch_url = "http://localhost:9200"
query_params = '{"size":2, "query" : {"match" : {"description": "Roman empire"}}}'
index_name = "historic_events"
whole_url = elasticsearch_url + "/" + index_name + "/_search"
header_json = {"Content-Type": "application/json"}
response = requests.get(whole_url, data=query_params, headers=header_json)
response.json()

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2142, 'relation': 'eq'},
  'max_score': 9.734037,
  'hits': [{'_index': 'historic_events',
    '_type': '_doc',
    '_id': 'M9EWansB2kiDSx5Po1tP',
    '_score': 9.734037,
    '_source': {'date': '172',
     'description': 'Montanism spreads through the Roman Empire.',
     'lang': 'en',
     'category1': 'By topic',
     'category2': 'Religion',
     'granularity': 'year'}},
   {'_index': 'historic_events',
    '_type': '_doc',
    '_id': 'V9EWansB2kiDSx5Po1ZJ',
    '_score': 9.469765,
    '_source': {'date': '-16',
     'description': 'Noricum is incorporated into the Roman Empire.',
     'lang': 'en',
     'category1': 'By place',
     'category2': 'Roman Empire',
     'granularity': 'year'}}]}}

Obdobným způsobem funguje i přidávání dokumentů do indexů, i jejich mazání.

In [10]:
elasticsearch_url = "http://localhost:9200"
new_document = '{"prvni_pole": 8, "druhe_pole": 800}'
index_name = "pokusny_index_2"
whole_url = elasticsearch_url + "/" + index_name + "/_doc"
header_json = {"Content-Type": "application/json"}
response = requests.post(whole_url, data=new_document, headers=header_json)
response.json()

{'_index': 'pokusny_index_2',
 '_type': '_doc',
 '_id': 'J-vapnsB2tj4EOjbwu4f',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 16,
 '_primary_term': 14}

In [11]:
elasticsearch_url = "http://localhost:9200"
document_id = "J-vapnsB2tj4EOjbwu4f"
index_name = "pokusny_index_2"
whole_url = elasticsearch_url + "/" + index_name + "/_doc/" + document_id
response = requests.delete(whole_url)
response.json()

{'_index': 'pokusny_index_2',
 '_type': '_doc',
 '_id': 'J-vapnsB2tj4EOjbwu4f',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 17,
 '_primary_term': 14}

## Kibana
#### Export obsahu indexu do csvčka
Pokud chceme převést obsah indexu do csv souboru, můžeme pro to použít Kibanu. Nejprve musíme v hlavním menu (tři vodorovné čáry vlevo nahoře) kliknout na "Discover" (sekce "Analytics").
![index_to_csv_1](elastic_figures/csv_1.png)
Následně vybereme index a jeho pole, které chceme do csvčka dostat. Máme přitom i možnost použít KQL (Kibaní dotazovací jazyk) - to se zapisuje do řádku vedle diskety. Například pokud chceme mít v csvčku pouze položky, u kterých bylo v poli "species" zapsáno "versicolor", napíšeme do tohoto řádku
```
species:"versicolor"
```
![index_to_csv_2](elastic_figures/csv_2.png)
Následně je třeba tuto věc uložit - bez toho vytvoření csvčka nebude umožněno. Posléze už lze pomocí tlačítka "Share" ->"CSV Reports" csvčko stáhnout.
![index_to_csv_3](elastic_figures/csv_3.png)